In [1]:
from johnsnowlabs import nlu
import pandas as pd
import itertools

In [2]:
pipe = nlu.load('glove')

glove_100d download started this may take some time.
Approximate size to download 145,3 MB
[OK!]


In [3]:
d = pd.read_csv('data/train/metrolyrics.csv')

In [5]:
for i in range(0, d.shape[0], 50):
    if i + 50 > d.shape[0]:
        j = d.shape[0]
    else:
        j = i + 50
    lyrics_embeddings = pipe.predict(d.lyrics[i:j], output_level='document')
    lyrics_embeddings.word_embedding_glove.apply(lambda x: list(itertools.chain(*x))).to_csv('data/train/emb_metrolyrics.csv', mode='a', index=False, header=False)